## Thread functions

In [1]:
#setup logging
import logging
logging.basicConfig(format='%(asctime)s:%(levelname)s:\t%(message)s', level=logging.DEBUG, datefmt='%m-%d %H:%M:%S')
logger = logging.getLogger('simple_example')
#logger.setLevel(logging.DEBUG)

pdebug = lambda x: logger.debug(x)
pinfo = lambda x: logger.info(x)
perror = lambda x: logger.error(x)
pexception = lambda x: logger.critical(x)



import threading
import time
from queue import Queue
from redis import Redis
import multiprocessing

exitFlag = 0

conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

# The base thread class to enable multithreading
class myThread (threading.Thread):
    def __init__(self, manager, name, callback, pubsub=True, msg=""):
        threading.Thread.__init__(self)
        self.threadID = manager.threadID
        self.name = name
        self.callback = callback
        self.pubsub = pubsub
        self.msg = ""
        self.manager = manager
        
    def run(self):
        pdebug("Starting " + self.name)
        if self.pubsub:
            self.thread_function(self.callback) # Runs an infinite thread
        else:
            self.thread_job(self.callback, self.msg)
        pdebug("Exiting " + self.name)  # Runs an thread job which terminates after completion
    
    # The thread function for infinite threads which can expect IPC using Redis
    def thread_function(self, callback):
        pubsub = conn.pubsub()
        pubsub.subscribe([self.name+'/cmd', self.name+'/msg'])
        
        pubsub.get_message(self.name+'/cmd')
        pubsub.get_message(self.name+'/msg')

        for item in pubsub.listen():
            channel = item['channel'].split('/')[1]
            msg = item['data']
            pdebug(self.name+':'+channel)
            if channel== 'msg':
                callback(self.manager, msg) # CALL: Thread callback function
            elif channel== 'cmd' and msg == 'stop':
                pubsub.unsubscribe() # Stop listening to the channel
                break
    
    # The thread function for one of tasks
    def thread_job(self, callback, msg):
        #pdebug(conn.rpop(queue))
        callback(msg) #CALL: Thread callback function with message queue

jobs = []
# Each thread manager runs in its own process
class threadManager():
    def __init__(self, name, thread_list, callback_list):
        #self.threads = []
        self.name = name
        self.threadList = thread_list
        self.threadCallback = callback_list
        
        self.start()
        
    def start(self):
        self.job = multiprocessing.Process(target=self.init)
        jobs.append(self.job)
        self.job.start() # Starting the process
        
        
    def init(self):
        self.threads = []
        self.threadID = 1
        
        # Create new threads
        for tName in self.threadList:
            self.add(tName, self.threadCallback[self.threadID-1])
        
        #pdebug("Init: waiting for threads to join")
        # Wait for all threads to complete
        for t in self.threads:
            t.join()
            #pdebug(".")
        pinfo("Exiting Main Thread")
        
    #TODO: Define function to restart a stopped thread
        
    def add(self, name, callback, pubsub=True, q="default"):
        # Create an instance of mythrade class and start the thread
        #pdebug("threadManager: add: before myThread: {}".format(name))
        self.thread = myThread(self, name, callback, pubsub, q)
        self.thread.start()
        #pdebug("threadManager: add: after myThread: {}".format(name))
        self.threads.append(self.thread)
        #pdebug(self.threads)
        self.threadID += 1


## Definition of handler functions

In [2]:
# This function is called by Kite or Kite_Simulation
def notification_despatcher(msg):
    pdebug('notification_despatcher: {}'.format(msg))
    # Step 1: Extract msg type: Tick/Callbacks
    
    Tick = True
    # Step 2.1: If Tick
    if Tick == True:
        # Push msg to msgBufferQueue
        conn.xadd('msgBufferQueue',{'msg': msg})
    
    # Step 2.2: else
    else:
        # Push msg to notificationQueue
        conn.xadd('notificationQueue',{'msg': msg})

# A thread function to process notifications and tick
def trade_job(msg):
    pdebug('trade_job: {}'.format(msg))
    
    # Step 1.1: Get stock name from the message
    
    # Step 1.2: Get state for the stock from the redis
    
    # Step 2: Switch to appropriate state machine based on current state
    
    # State: Init
        # 1: Populate Redis buffer stock+"OHLCBuffer" with historical data
        
        # 2: Set state to Scanning
    
    # State: Scanning
        # 1: Run trading algorithm for entering trade
        
        # 2: If Algo returns Buy: set State to 'Pending Order: Long'
        
        # 3: If Algo returns Sell: set State to 'Pending Order: Short'
        
        # 4: Update TradeMetaData: Push order details to OrderQueue
    
    # State: Pending Order: Long
        # 1: On Fill: set State to Long
    
    # State: Pending Order: Short
        # 1: On Fill: set State to Short
    
    # State: Long
        # 1: If notification for AutoSquare Off: set state to init
        
        # 2: Else run trading algorithm for square off
        
        # 3: If algo returns square off: then push square off details to OrderQueue, set state to 'Awaiting Square Off'
    
    # State: Short
        # 1: If notification for AutoSquare Off: set state to init
        
        # 2: Else run trading algorithm for square off
        
        # 3: If algo returns square off: then push square off details to OrderQueue, set state to 'Awaiting Square Off'
    
    # State: Awaiting Square Off
        # 1: On Fill notification: set state to Init
    
    
def trade_handler(manager, msg):
    pdebug('trade_handler: {}'.format(msg))
    # Step 1: Blocking call to userRequestsQueue, msgBufferQueue and notificationQueue
    
    # Step 3: Process userRequest: Start a worker thread for each request
    
    # Step 3: Process notifications: Start a worker thread for each notification
    
    # Step 4: Process tick: Start a worker thread for each msg
    
    
def order_handler(manager, msg):
    pdebug('order_handler: {}'.format(msg))
    
    # Step 1: Block for new order request: OrderQueue
    
    # Step 2: Create order msg for Kite: fill metadata
    
    # Step 3: If papertrade: create a log entry
    
    # Step 4: If not a papertrade: despatch order
    
def backtest_handler(manager, msg):
    pdebug('backtest_handler: {}'.format(msg))
    # Start an interval thread: 1000 ms
    
    # Calculate trade status data, charts and analytics
    
    # Update redis cache with figure and msg
    
def kite_simulator(manager, msg):
    pdebug('kite_simulator: {}'.format(msg))
    # Loop through OHLC data from local storage
    
    # Check square off conditions
    
    # Construct Json message like Kite
    
    # Call notification_despatcher
    
    # Optional: wait few miliseconds
    

In [3]:
freedom = threadManager("freedom", ["kite_simulator", "backtest_handler", "trade_handler","order_handler"], 
                        [kite_simulator, backtest_handler, trade_handler, order_handler])

04-11 20:33:33:DEBUG:	Starting kite_simulator
04-11 20:33:33:DEBUG:	Starting backtest_handler
04-11 20:33:33:DEBUG:	Starting trade_handler
04-11 20:33:33:DEBUG:	Starting order_handler
04-11 20:33:33:DEBUG:	kite_simulator:msg
04-11 20:33:33:DEBUG:	backtest_handler:msg
04-11 20:33:33:DEBUG:	order_handler:cmd
04-11 20:33:33:DEBUG:	kite_simulator: 2
04-11 20:33:33:DEBUG:	backtest_handler: 2
04-11 20:33:33:DEBUG:	order_handler:msg
04-11 20:33:33:DEBUG:	order_handler: 2
04-11 20:34:12:DEBUG:	kite_simulator:msg
04-11 20:34:12:DEBUG:	kite_simulator: Hello Dear
04-11 20:34:43:DEBUG:	kite_simulator:msg
04-11 20:34:43:DEBUG:	kite_simulator: Hello Dear
04-11 20:34:43:DEBUG:	kite_simulator:cmd
04-11 20:34:43:DEBUG:	Exiting kite_simulator


In [4]:
freedom.job
freedom.job.is_alive()

True

In [8]:
pdebug(conn.pubsub_channels())
conn.pubsub_numsub('kite_simulator/msg')
conn.publish('kite_simulator/msg','Hello Dear')
conn.publish('kite_simulator/cmd','stop')

04-11 20:34:43:DEBUG:	['backtest_handler/cmd', 'backtest/cmd', 'trade_handler/msg', 'order_handler/cmd', 'trade_handler/cmd', 'kite_simulator/msg', 'backtest_handler/msg', 'kite_simulator/cmd', 'backtest/data', 'order_handler/msg']


1

In [84]:
#from walrus import Database  # A subclass of the redis-py Redis client.

#db = Database(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)
#stream = conn.Stream('stream-a')

In [66]:
msgid = conn.xadd('stream-1',{'message': 'hello, streams'})
print(msgid)

1586632321501-0


In [45]:
msgid2 = conn.xadd('stream-1',{'message': 'message 2'})
msgid3 = conn.xadd('stream-1',{'message': 'message 3'})

In [58]:
msgid2 = conn.xadd('stream-1',{'message': 'message 4'})

In [104]:
conn.xtrim('stream-1',maxlen=0, approximate=False)

5

In [105]:
help(conn.xtrim)

Help on method xtrim in module redis.client:

xtrim(name, maxlen, approximate=True) method of redis.client.Redis instance
    Trims old messages from a stream.
    name: name of the stream.
    maxlen: truncate old stream messages beyond this size
    approximate: actual stream length may be slightly more than maxlen



In [106]:
conn.xrange('stream-1')

[]

In [ ]:
import numpy as np
import time
for i in np.linspace(1,5,5):
    msg = conn.xread({'stream-1':'$','stream-2':'$'}, block=0, count=100)
    msg = conn.xread({'stream-1':'0','stream-2':'0'}, block=5000, count=100)
    pdebug(msg)

In [32]:
messages = stream[msgid::1]
print(messages)

[('1586629344621-0', {'message': 'hello, streams'})]


In [37]:
list(stream)

[('1586629344621-0', {'message': 'hello, streams'}),
 ('1586629368816-0', {'message': 'message 2'}),
 ('1586629368818-0', {'message': 'message 3'})]

In [36]:
# Adding and deleting a message:
msgid4 = stream.add({'message': 'delete me'})
del stream[msgid4]

# How many items are in the stream?
print(len(stream))  # Prints 3.

3


In [38]:
# Add 1000 items to "stream-2".
stream2 = db.Stream('stream-2')
for i in range(1000):
    stream2.add({'data': 'message-%s' % i})

# Trim stream-2 to (approximately) 10 most-recent messages.
nremoved = stream2.trim(10)
print(nremoved)
# 909
print(len(stream2))
# 91

# To trim to an exact number, specify `approximate=False`:
stream2.trim(10, approximate=False)  # Returns 81.
print(len(stream2))

900
100
10


In [40]:
stream.read(timeout=2000, last_id='$')

TypeError: read() got an unexpected keyword argument 'timeout'